# **Launch Sites Locations Analysis with Folium**

Using Folium, we design visual analytics, which would help us understand the geographic impact on the success rate for the launch site locations. There are three steps to creating this interactive map of the launch sites; we first mark all launch sites on the map. Secondly, we label each launch site's failed and successful launch outcomes in their respective areas. Finally, calculate the distance between the launch site and different proximities such as; railway connections, highways, coastlines, and cities.

In [1]:
# Install Libraries
# !pip3 install folium==0.7.0
# !pip3 install wget 

In [2]:
# Import Libraries
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

### Mark all Launch SItes on a Map 

In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [8]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i in launch_sites_df.index:
    coordinate = [launch_sites_df.loc[i, 'Lat'], launch_sites_df.loc[i, 'Long']]
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(launch_sites_df.loc[i, 'Launch Site']))
    marker = folium.map.Marker(coordinate, 
                      icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), 
                                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df.loc[i, 'Launch Site'], ))
    site_map.add_child(circle)
    site_map.add_child(marker)
    
site_map

### Mark the success/failed launches for each site on the map

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [10]:
# Create a marker cluster since we have many
marker_cluster = MarkerCluster()

In [11]:
spacex_df = spacex_df.rename(columns={'class': 'Class'})
# A function to check the value of `class` column
def check_class(Class):
    if Class == 1:
        return 'green'
    else: 
        return 'red'
    
launch_sites_df['marker_color'] = spacex_df['Class'].apply(check_class)
launch_sites_df.tail(10)

,Launch Site,Lat,Long,marker_color
0,CCAFS LC-40,28.562302,-80.577356,red
1,CCAFS SLC-40,28.563197,-80.576820,red
2,KSC LC-39A,28.573255,-80.646895,red
3,VAFB SLC-4E,34.632834,-120.610746,red


In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,Class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
#icon = folium.Icon(color='white', icon_color=row['marker_color'])
                   
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    coordinate = [record[1], record[2]]
    marker = folium.Marker(coordinate, 
                      icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), 
                                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % record[0], ))
    marker_cluster.add_child(marker)

site_map

### Calculate the distances between a launch site to its proximities

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
# find coordinate of the closet coastline
coastline_lat, coastline_lon = 28.56367, -80.57163
launch_site_lat, launch_site_lon = 28.57, -80.64
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

6.715985674083324

In [17]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
coordinate = coastline_lat, coastline_lon
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

In [18]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = ([launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon])
lines = folium.PolyLine(locations = coordinates, weight = 1)
site_map.add_child(lines)

In [19]:
#Nearest City Orlando
coordinate_NC = [28.53659, -81.38122]

circle = folium.Circle(coordinate_NC, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Orlando'))
marker_NC = folium.map.Marker(coordinate_NC, 
                      icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), 
                                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Orlando City', ))
site_map.add_child(marker_NC)
site_map.add_child(circle)

coordinates_city = ([launch_site_lat, launch_site_lon], coordinate_NC)
lines = folium.PolyLine(locations = coordinates_city, weight = 1)
site_map.add_child(lines)

distance_city = calculate_distance(launch_site_lat, launch_site_lon, 28.53659, -81.38122)
coordinate_dis = [28.53853, -81.32743]
distance_marker = folium.Marker(
    coordinate_dis,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
site_map.add_child(distance_marker)